In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

## 데이터 불러오기

In [2]:
# pickle 파일 불러오기
with open('/home/sjkim/추천공모전/data/export_GDP_pop_fx.pkl', 'rb') as f:
    ratings_all = pickle.load(f)

In [3]:
ratings_all

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY
0,2012,Afghanistan,3.0,1.0,2.020357e+07,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
1,2012,Afghanistan,4.0,0.0,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
2,2012,Afghanistan,7.0,2.0,2.020357e+07,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
3,2012,Afghanistan,8.0,0.0,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
4,2012,Afghanistan,10.0,12.0,2.020357e+07,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116390,2021,Zimbabwe,87.0,1289.0,2.837124e+07,4.543334e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56
116391,2021,Zimbabwe,90.0,322.0,2.837124e+07,1.134952e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56
116392,2021,Zimbabwe,94.0,1.0,2.837124e+07,3.524696e-08,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56
116393,2021,Zimbabwe,95.0,13.0,2.837124e+07,4.582105e-07,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56


In [4]:
ratings = ratings_all[ratings_all['year']==2012]
ratings = ratings[['country', '품목코드', 'preference']]
# column명 변경
ratings.columns=['country_id', 'item_id', 'preference']

# country에 고유번호 부여하기
country_to_idx = {v:k for k,v in enumerate(ratings['country_id'].unique())}
idx_to_country = {v:k for k,v in country_to_idx.items()}

# country_id를 고유번호로 바꾸기
ratings['country_id'] = ratings['country_id'].map(country_to_idx)

# item_id int로 바꾸기
ratings['item_id'] = ratings['item_id'].astype(int)

# 중복제거
ratings = ratings.drop_duplicates(['country_id', 'item_id'], keep='first')

# index reset
ratings.reset_index(drop=True, inplace=True)
ratings


,country_id,item_id,preference
0,0,3,4.949620e-08
1,0,4,0.000000e+00
2,0,7,9.899239e-08
3,0,8,0.000000e+00
4,0,10,5.939543e-07
...,...,...,...
11021,202,91,0.000000e+00
11022,202,94,0.000000e+00
11023,202,95,8.764319e-07
11024,202,96,1.869721e-06


In [5]:
# train-test set 나누기
TRAIN_SIZE = 0.75                             # train 75: valid 25 비율로 분리
ratings = shuffle(ratings, random_state=12)   # train set을 무작위로 섞는다. (국가-품목-선호도)가 하나의 세트이기 때문에 섞어도 문제 없음
cutoff = int(TRAIN_SIZE * len(ratings))       # 전체 데이터 중 train_size의 비율에 해당하는 데이터가 몇 개인지 계산
ratings_train = ratings.iloc[:cutoff]           
ratings_test = ratings.iloc[cutoff:]

## MF

In [6]:
# New MF class for training & testing
class NEW_MF():
    # Initializing the object
    def __init__(self, ratings, K, alpha, beta, iterations, tolerance=0.005, verbose=True):
        self.R = np.array(ratings)                        # R에 개의 데이터 존재

        # 국가 아이디, 아이템 아이디가 R(내부)의 인덱스와 일치하지 않을 수 있음. 매칭하기 위해 dictionary 사용
        item_id_index = []                                  
        index_item_id = []
        for i, one_id in enumerate(ratings):                # ratings의 각 아이템에 대해서 아래 작업 수행.
            item_id_index.append([one_id, i])               # 아이디를 인덱스로 매핑. item_id_index에 현재 아이템의 id와 index를 저장한다. columns값이 one_id로 들어감!
            index_item_id.append([i, one_id])               # 인덱스를 아이디로 매핑. index_item_id에 현재 아이템의 index와 id를 저장한다
        self.item_id_index = dict(item_id_index)            # dictionary 형태로 변환
        self.index_item_id = dict(index_item_id)        
        country_id_index = []                               # 똑같은 작업을 country_id에 대해서도 실행
        index_country_id = []
        for i, one_id in enumerate(ratings.T):
            country_id_index.append([one_id, i])
            index_country_id.append([i, one_id])
        self.country_id_index = dict(country_id_index)
        self.index_country_id = dict(index_country_id)
        # 다른 변수 초기화
        self.num_country, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.tolerance = tolerance
        self.verbose = verbose

    # 테스트 셋을 선정하는 메소드 
    def set_test(self, ratings_test):                           # 분리된 test set을 넘겨받아서 클래스 내부의 test set을 만드는 함수
        test_set = []
        for i in range(len(ratings_test)):                      # 테스트 데이터에서 각 (국가-아이템-선호도)에 대해서 아래 작업 반복
            x = self.country_id_index[ratings_test.iloc[i,0]]   # 현재 국가의 인덱스를 country_id_index 받아온다
            y = self.item_id_index[ratings_test.iloc[i,1]]      # 현재 아이템의 인덱스를 item_id_index에서 받아온다
            z = ratings_test.iloc[i,2]                          # 현재 국가-아이템의 선호도를 받아온다
            test_set.append([x, y, z])                          # 현재 (국가-아이템-선호도)를 test_set 리스트에 추가한다
            self.R[x, y] = 0                                    # (국가-아이템-선호도)를 R에서 0으로 지운다. test set을 제거해야 함
        self.test_set = test_set                                # test_set을 클래스에 저장한다
        return test_set                                         # Return test set

    def test(self):                                             # Training 하면서 test set의 정확도를 계산하는 메소드
        # Initializing country-feature and item-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_country, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_country)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()                                    # R에서 평점이 있는(0이 아닌) 요소의 인덱스를 가져온다. train set(set_test() 함수에서 test set에 해당하는 부분은 지웠기 때문에(0으로 바꿨기 때문에) R전체가 train set이 된다)
        self.samples = [(i,j, self.R[i,j]) for i, j in zip(rows, columns)]  # train set(R)에 대해서 (국가-아이템-선호도) 데이터를 구성한다.

        # Stochastic gradient descent for given number of iterations
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        for i in range(self.iterations):                                    # 지정된 반복 횟수만큼 아래 코드 실행
            np.random.shuffle(self.samples)                                 # 데이터 섞기. SGD를 어디에서 시작하느냐에 따라 수렴의 속도가 달라질 수 있기 때문에 매 반복마다 다양한 시작점에서 출발하기 위함
            self.sgd()                                                      # SGD 방법으로 P, Q, bu, bd 업데이트
            rmse1 = self.rmse()                                             # train set의 rmse
            rmse2 = self.test_rmse()                                        # test set의 rmse
            training_process.append((i+1, rmse1, rmse2))                    # iteration의 수, train rmse, test rmse를 결과에 추가
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                               # New best record
                best_RMSE = rmse2
                best_iteration = i
            elif (rmse2 - best_RMSE) > self.tolerance:          # RMSE is increasing over tolerance. 차이가 tolerance보다 커지면 break -> ealry stopping같은 개념
                break
        print(best_iteration, best_RMSE)
        return training_process

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])

    # Computing mean squared error
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Test set에 대한 예측치를 계산해서 test set에 대한 RMSE를 계산하는 method 
    def test_rmse(self):
        error = 0                                                       # 에러 초기화
        for one_set in self.test_set:                                   # test set에 있는 (국가-아이템-선호도)에 대해서 아래 작업 실행
            predicted = self.get_prediction(one_set[0], one_set[1])     # get_grediction()함수를 불러서 country_id의 item_id에 대한 예측 평점을 받아온다
            error += pow(one_set[2] - predicted, 2)                     # one_set[2]가 현재 (국가-아이템-선호도)에서의 실제 선호도 값이고 predicted가 예측치임! error에는 오차 제곱한 것을 누적한 값이 저장됨
        return np.sqrt(error/len(self.test_set))                        # error를 rmse로 변환해서 돌려준다

    # Ratings for country i and item j                                  # 모든 국가의 모든 아이템에 대한 예측치(full matrix)를 계산해서 돌려준다
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Ratings for country_id and item_id
    def get_one_prediction(self, country_id, item_id):                    # 주어진 country_id, item_id에 대한 예측값을 돌려준다
        return self.get_prediction(self.country_id_index[country_id], self.item_id_index[item_id])
    
    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis:,] + self.P.dot(self.Q.T)

In [7]:
# Testing MF RMSE
R_temp = ratings.pivot(index='country_id', columns='item_id', values='preference').fillna(0)
mf = NEW_MF(R_temp, K=220, alpha=0.0014, beta=0.075, iterations=350, tolerance=0.0001, verbose=True)
test_set = mf.set_test(ratings_test)                    # ratings_test를 test 데이터로 저장하도록 set_test()함수 호출
result = mf.test()                                      # 정해진 파라미터에 따라 MF 훈련과 정확도 계산 실행 print(best_iteration, best_RMSE)

Iteration: 10 ; Train RMSE = 0.002949 ; Test RMSE = 0.005585
Iteration: 20 ; Train RMSE = 0.002911 ; Test RMSE = 0.005560
Iteration: 30 ; Train RMSE = 0.002886 ; Test RMSE = 0.005554
Iteration: 40 ; Train RMSE = 0.002867 ; Test RMSE = 0.005555
Iteration: 50 ; Train RMSE = 0.002852 ; Test RMSE = 0.005559
Iteration: 60 ; Train RMSE = 0.002840 ; Test RMSE = 0.005564
Iteration: 70 ; Train RMSE = 0.002831 ; Test RMSE = 0.005569
Iteration: 80 ; Train RMSE = 0.002823 ; Test RMSE = 0.005575
Iteration: 90 ; Train RMSE = 0.002816 ; Test RMSE = 0.005581
Iteration: 100 ; Train RMSE = 0.002811 ; Test RMSE = 0.005586
Iteration: 110 ; Train RMSE = 0.002806 ; Test RMSE = 0.005591
Iteration: 120 ; Train RMSE = 0.002803 ; Test RMSE = 0.005595
Iteration: 130 ; Train RMSE = 0.002800 ; Test RMSE = 0.005600
Iteration: 140 ; Train RMSE = 0.002797 ; Test RMSE = 0.005604
Iteration: 150 ; Train RMSE = 0.002795 ; Test RMSE = 0.005608
Iteration: 160 ; Train RMSE = 0.002794 ; Test RMSE = 0.005611
Iteration: 170 ; 

In [11]:
print(mf.full_prediction())

[[ 6.70645813e-05 -2.83444376e-05  4.92492676e-04 ...  1.11839107e-04
   8.61169109e-05  1.89986248e-04]
 [-2.51298016e-04 -2.06973218e-04  2.59038976e-04 ... -9.88275195e-05
  -1.17097901e-04 -4.09464672e-05]
 [-8.33240075e-05 -2.23421173e-04  3.24237966e-04 ... -5.38656264e-05
  -7.58382083e-05 -2.62729070e-05]
 ...
 [-4.15114467e-05 -1.54094627e-04  3.61397479e-04 ... -2.81147229e-05
  -5.10434861e-05  1.19241120e-05]
 [-7.57714060e-05 -3.67299600e-04  1.90465286e-04 ... -1.60465449e-04
  -1.85009700e-04 -2.00187310e-04]
 [-1.40686072e-04 -2.69459158e-04  2.41553684e-04 ... -1.23080118e-04
  -1.47576978e-04 -1.32101305e-04]]


In [12]:
print(mf.get_one_prediction(1,5), R_temp.loc[1][2])      # 1번 국가 2번 아이템 예측값, 실제값 출력

-6.735945970752783e-05 0.0


## MF로 추천하기

In [13]:
#### 전체 데이터 불러오기 ####
ratings = ratings_all[ratings_all['year']==2012]
ratings = ratings[['country', '품목코드', 'preference']]
# column명 변경
ratings.columns=['country_id', 'item_id', 'preference']

# country에 고유번호 부여하기
country_to_idx = {v:k for k,v in enumerate(ratings['country_id'].unique())}
idx_to_country = {v:k for k,v in country_to_idx.items()}

# country_id를 고유번호로 바꾸기
ratings['country_id'] = ratings['country_id'].map(country_to_idx)

# item_id int로 바꾸기
ratings['item_id'] = ratings['item_id'].astype(int)

# 중복제거
ratings = ratings.drop_duplicates(['country_id', 'item_id'], keep='first')

# index reset
ratings.reset_index(drop=True, inplace=True)

rating_matrix = ratings.pivot(values='preference', index='country_id', columns='item_id')

# 품목 이름 가져오기
items_df = pd.read_csv('/home/sjkim/추천공모전/data/item_name-code.csv')
items_df.columns=['title', 'item_id']
items_df = items_df.set_index('item_id')

In [14]:
# 추천하기
def recommender(country, n_items=10):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    rated_index = rating_matrix.loc[country][rating_matrix.loc[country] > 0].index    # 선호도가 존재하는 품목 확인
    items = rating_matrix.loc[country].drop(rated_index)
    for item in items.index:
        predictions.append(mf.get_one_prediction(country, item))                      # 예상 선호도 계산
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]          # 예상 선호도가 가장 높은 품목 선택
    recommended_items = items_df.loc[recommendations.index]['title']
    return recommended_items

In [15]:
# 추천 함수 부르기
recommender(1, 5)

item_id
89                                    선박과 수상 구조물
27    광물성 연료ㆍ광물유(鑛物油)와 이들의 증류물, 역청(瀝靑)물질, 광물성 왁스
72                                            철강
60                               메리야스 편물과 뜨개질 편물
3                 어류ㆍ갑각류ㆍ연체동물과 그 밖의 수생(水生) 무척추동물
Name: title, dtype: object

## Precision@K

In [16]:
# precision@k 함수로 만들기
def precision_at_k(country, n_items, year):
    hit_df = ratings_all[ratings_all['year'].isin(year)]
    hit_df = hit_df[hit_df['country']==idx_to_country[country]]
    c_items = hit_df['품목코드'].unique().astype(int)
    rec_items = recommender(country, n_items).index.tolist()
    hit_items = [i for i in c_items if i in rec_items]
    precision = len(hit_items) / n_items
    return precision

In [18]:
# 2012년 모든 나라에 대해서 precision@k 계산
def precision_at_k_all(n_items, year):
    precision = []
    c_len = len(rating_matrix)
    for i in range(0, c_len):
        precision.append(precision_at_k(i, n_items, year))
    return sum(precision) / c_len


year = [2013, 2014, 2015, 2016, 2017]
precision_at_k_all(5, year)

0.7123152709359606